##Core

In [ ]:
!pip install fastcore fastai2 tokenizers -q

     |████████████████████████████████| 194kB 11.9MB/s 
     |████████████████████████████████| 3.8MB 28.2MB/s 


In [ ]:
from fastcore.foundation import *
from fastai2.text.all import *
# import pathlib

In [ ]:
# @patch
# def ls(x: pathlib.Path):
#     return list(x.iterdir())

## Colab_setup

In [ ]:
import os

from pathlib import Path
from google.colab import drive

In [ ]:
def create_path(path):
    if not os.path.isdir(path):
        path.mkdir(parents=True, exist_ok=True)
    return path

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
root_dir = Path('/content/drive/My Drive')
base_path = create_path(root_dir/'seq2seq')
base_path

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Path('/content/drive/My Drive/seq2seq')

In [ ]:
colab_path = Path('/content')

In [ ]:
data_path = create_path(base_path/'dataset')

In [ ]:
model_path = create_path(base_path/'models')

## Imports

In [ ]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import numpy as np
# import pandas as pd
from tokenizers import ByteLevelBPETokenizer

##Get data

In [ ]:
df = pd.read_csv(data_path/'en-fr-questions.csv'); df.head()

,eng,fr
0,What is light ?,Qu’est-ce que la lumière?
1,Who are we?,Où sommes-nous?
2,Where did we come from?,D'où venons-nous?
3,What would we do without it?,Que ferions-nous sans elle ?
4,"What is the absolute location (latitude and longitude) of Badger, Newfoundland and Labrador?","Quelle sont les coordonnées (latitude et longitude) de Badger, à Terre-Neuve-etLabrador?"


In [ ]:
df.eng.to_csv(colab_path/'temp_eng.txt', header=None, index=None, sep=' ', mode='a')
df.fr.to_csv(colab_path/'temp_fr.txt', header=None, index=None, sep=' ', mode='a')

#tokenize text

I will be using the tokenizers library from huggingface to tokenize both the english and french texts using a BPE to avoid unknown tokens. SInce we are using a Bytelevel tokenizer, we have to train it on our own corpus for it to create it's vocab

references: https://github.com/huggingface/tokenizers

In [ ]:
tokenizer_eng = ByteLevelBPETokenizer()
tokenizer_eng.train(str(colab_path/'temp_eng.txt'), special_tokens=['<sos>', '<eos>', '<pad>'])

In [ ]:
tokenizer_fr = ByteLevelBPETokenizer()
tokenizer_fr.train(str(colab_path/'temp_fr.txt'), special_tokens=['<sos>', '<eos>', '<pad>'])

In [ ]:
tokenizer_eng.save(str(data_path), 'bpe_eng_vocab')
tokenizer_fr.save(str(data_path), 'bpe_fr_vocab')

['/content/drive/My Drive/seq2seq/dataset/bpe_fr_vocab-vocab.json',
 '/content/drive/My Drive/seq2seq/dataset/bpe_fr_vocab-merges.txt']

In [ ]:
tokenizer_eng = ByteLevelBPETokenizer(str(data_path/'bpe_eng_vocab-vocab.json'),
                                      str(data_path/'bpe_eng_vocab-merges.txt'))

tokenizer_fr = ByteLevelBPETokenizer(str(data_path/'bpe_fr_vocab-vocab.json'),
                                      str(data_path/'bpe_fr_vocab-merges.txt'))

In [ ]:
vocab_size_eng = tokenizer_eng.get_vocab_size()
vocab_size_fr = tokenizer_fr.get_vocab_size()

print(vocab_size_eng, vocab_size_fr)

23077 28091


In [ ]:
# PAD_TOKEN = tokenizer_fr.encode('<pad>').ids[0]; PAD_TOKEN
PAD_TOKEN_ID = 2

In [ ]:
vocab_fr = [k for k, v in sorted(tokenizer_fr.get_vocab().items(), key=lambda x: x[1])]
vocab_eng = [k for k, v in sorted(tokenizer_eng.get_vocab().items(), key=lambda x: x[1])]

In [ ]:
print(vocab_eng)

['<sos>', '<eos>', '<pad>', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '¡', '¢', '£', '¤', '¥', '¦', '§', '¨', '©', 'ª', '«', '¬', '®', '¯', '°', '±', '²', '³', '´', 'µ', '¶', '·', '¸', '¹', 'º', '»', '¼', '½', '¾', '¿', 'À', 'Á', 'Â', 'Ã', 'Ä', 'Å', 'Æ', 'Ç', 'È', 'É', 'Ê', 'Ë', 'Ì', 'Í', 'Î', 'Ï', 'Ð', 'Ñ', 'Ò', 'Ó', 'Ô', 'Õ', 'Ö', '×', 'Ø', 'Ù', 'Ú', 'Û', 'Ü', 'Ý', 'Þ', 'ß', 'à', 'á', 'â', 'ã', 'ä', 'å', 'æ', 'ç', 'è', 'é', 'ê', 'ë', 'ì', 'í', 'î', 'ï', 'ð', 'ñ', 'ò', 'ó', 'ô', 'õ', 'ö', '÷', 'ø', 'ù', 'ú', 'û', 'ü', 'ý', 'þ', 'ÿ', 'Ā', 'ā', 'Ă', 'ă', 'Ą', 'ą', '

In [ ]:
tokenizer_eng.encode('Tokenizing is so easy').tokens

['T', 'ok', 'en', 'izing', 'Ġis', 'Ġso', 'Ġeasy']

## Building the model

In [ ]:
# class PositionalEncoding(nn.Module):
#     def __init__(self, emb_dim, dropout, max_len=5000):
#         super().__init__()
#         self.dropout = nn.Dropout(dropout)
#         pe = torch.zeros(max_len, emb_dim)
#         position = torch.arange(0, max_len, 1, dtype=torch.float).unsqueeze(1)
#         div_term = torch.exp(torch.arange(0, emb_dim, 2, dtype=torch.float) * (-math.log(10000.0) / emb_dim))
#         pe[:, 0::2] = torch.sin(position * div_term) #[max_len, emb_dim]
#         pe[:, 1::2] = torch.cos(position * div_term) #[max_len, emb_dim]

#         # store the pe in the persistent buffer so they won't be discarded and 
#         # also they won't be thought of as params
#         self.register_buffer('pe', pe)

#     def forward(self, inp_emb):
#         inp_emb = inp_emb + self.pe[inp_emb.shape[0], :]
#         inp_emb = self.dropout(inp_emb)
#         return inp_emb

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, emb_dim, dropout, max_len=5000):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        # pe = torch.zeros(max_len, emb_dim)
        # position = torch.arange(0, max_len, 1, dtype=torch.float).unsqueeze(1)
        # div_term = torch.exp(torch.arange(0, emb_dim, 2, dtype=torch.float) * (-math.log(10000.0) / emb_dim))
        # pe[:, 0::2] = torch.sin(position * div_term) #[max_len, emb_dim]
        # pe[:, 1::2] = torch.cos(position * div_term) #[max_len, emb_dim]

        # store the pe in the persistent buffer so they won't be discarded and 
        # also they won't be thought of as params
        self.register_buffer('freq', 1 / (10000 ** (torch.arange(0., emb_dim, 2.)/emb_dim)))

    def forward(self, inp_emb):
        pos = torch.arange(0, inp_emb.size(1), device=inp_emb.device).float()
        inp = torch.ger(pos, self.freq)
        pos_enc = torch.cat([inp.sin(), inp.cos()], dim=-1)

        inp_emb = inp_emb + pos_enc
        inp_emb = self.dropout(inp_emb)
        return inp_emb

In [ ]:
PositionalEncoding(300, 0.1)(torch.randn(32, 16, 300)).shape

torch.Size([32, 16, 300])

Since we have a translation(next word problem), it is the decoder that will be masked. For more info on the kep_pads reference https://discuss.pytorch.org/t/how-to-add-padding-mask-to-nn-transformerencoder-module/63390/3

In [ ]:
class TransformerModel(nn.Module):
    # adapted from 
    def __init__(self, enc_voc_sz, dec_voc_sz, emb_sz, n_head, inp_fc, num_layers, device, dropout=0.5):
        super(TransformerModel, self).__init__()
        self.device = device

        self.emb_sz = emb_sz

        self.enc_emb = nn.Embedding(enc_voc_sz, emb_sz).to(self.device)
        self.dec_emb = nn.Embedding(dec_voc_sz, emb_sz).to(self.device)

        trans_enc_layer = nn.TransformerEncoderLayer(emb_sz, nhead=n_head, dim_feedforward=inp_fc, dropout=dropout).to(self.device)
        self.trans_enc = nn.TransformerEncoder(trans_enc_layer, num_layers=num_layers).to(self.device)
        self.pos_enc = PositionalEncoding(emb_sz, dropout).to(self.device)

        # self.dec_inp_mask = None

        trans_dec_layer = nn.TransformerDecoderLayer(emb_sz, nhead=n_head, dim_feedforward=inp_fc, dropout=dropout).to(self.device)
        self.trans_dec = nn.TransformerDecoder(trans_dec_layer, num_layers=num_layers).to(self.device)
        self.pos_dec = PositionalEncoding(emb_sz, dropout).to(self.device)

        self.decout = nn.Linear(emb_sz, dec_voc_sz).to(self.device)
        self.decout.weight = self.dec_emb.weight

        # self.init_weight()
        

    def get_mask(self, sz):
        mask = torch.triu(torch.ones(sz, sz)).transpose(0, 1)
        mask = mask.masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask.to(self.device)

    def get_src_key_pad(self, src):
        # a boolean is expected for key masks to indicate padding
        src = (src == PAD_TOKEN_ID).long().bool()
        # src = (src == PAD_TOKEN_ID).float()
        # src = src.masked_fill(src==1., float('-inf'))
        return src.to(self.device)

    def get_trg_key_and_mask(self, src):
        trg_mask = self.get_mask(len(src))
        trg_key_pad_mask = self.get_src_key_pad(src)
        return trg_mask, trg_key_pad_mask


    def forward(self, enc_inp, dec_inp):
        ###########################encoder############################
        # according to attention is all you need paper the encoder embedding 
        # must be multplied by the sqrt of the embedding_size

        # ensure the input is of shape [seqlen, bs]
        # if not run enc_inp = enc_inp.t()
        enc_inp = enc_inp.t().to(self.device)
        enc_key_pad_mask = self.get_src_key_pad(enc_inp) #[seq_len, bs]

        # print(f'enc_emb: {self.enc_emb.device}')
        # print(f'enc_inp: {enc_inp.device}')
        # print(f'sqrt: {math.sqrt(self.emb_sz).device}')
        enc_inp = self.enc_emb(enc_inp) * math.sqrt(self.emb_sz)
        enc_inp = self.pos_enc(enc_inp)
        
        # ensure the input is of shape [seqlen, bs, emb_sz] after emb_enc and pos_enc
        enc_out = self.trans_enc(src=enc_inp, src_key_padding_mask=enc_key_pad_mask.t()) #[seq_len, bs, emb_sz]
        
        ###########################decoder############################
        # ensure the input is of shape [seqlen, bs]
        # if not run dec_inp = dec_inp.t()
        dec_inp = dec_inp.t().to(self.device)
        dec_mask, dec_key_pad_mask = self.get_trg_key_and_mask(dec_inp)

        dec_inp = self.dec_emb(dec_inp) * math.sqrt(self.emb_sz)
        dec_inp = self.pos_dec(dec_inp)
        
        # ensure the input is of shape [seqlen, bs, emb_sz] after emb_enc and pos_enc
        dec_out = self.trans_dec(tgt=dec_inp, memory=enc_out, tgt_mask=dec_mask, 
                                 tgt_key_padding_mask=dec_key_pad_mask.t(), 
                                 memory_key_padding_mask=enc_key_pad_mask.t())
        
        dec_out  = self.decout(dec_out)

        return dec_out

    def init_weight(self):
        # it is important that the distrbution of the weights be mean and std 0
        initRange = 0.05
        self.enc_emb.weight.data.uniform_(-initRange, initRange)
        self.dec_emb.weight.data.uniform_(-initRange, initRange)

        self.decout.weight.data.uniform_(-initRange, initRange)
        self.decout.bias.data.zero_()

    def eval(self):
        self.trans_enc.eval()
        self.trans_dec.eval()

In [ ]:
class TransformerModel(nn.Module):
    # adapted from 
    def __init__(self, enc_voc_sz, dec_voc_sz, emb_sz, n_head, inp_fc, num_layers, device, dropout=0.5):
        super(TransformerModel, self).__init__()
        self.device = device

        self.emb_sz = emb_sz

        self.enc_emb = nn.Embedding(enc_voc_sz, emb_sz).to(self.device)
        self.dec_emb = nn.Embedding(dec_voc_sz, emb_sz).to(self.device)

        trans_enc_layer = nn.TransformerEncoderLayer(emb_sz, nhead=n_head, dim_feedforward=inp_fc, dropout=dropout).to(self.device)
        self.trans_enc = nn.TransformerEncoder(trans_enc_layer, num_layers=num_layers).to(self.device)
        self.pos_enc = PositionalEncoding(emb_sz, dropout).to(self.device)

        # self.dec_inp_mask = None

        trans_dec_layer = nn.TransformerDecoderLayer(emb_sz, nhead=n_head, dim_feedforward=inp_fc, dropout=dropout).to(self.device)
        self.trans_dec = nn.TransformerDecoder(trans_dec_layer, num_layers=num_layers).to(self.device)
        # self.pos_dec = PositionalEncoding(emb_sz, dropout).to(self.device)

        self.decout = nn.Linear(emb_sz, dec_voc_sz).to(self.device)

        self.init_weight()

        self.pos_enc = nn.Embedding(enc_voc_sz, 512)
        self.pos_dec = nn.Embedding(dec_voc_sz, 512)

    def get_mask(self, sz):
        mask = torch.triu(torch.ones(sz, sz)).transpose(0, 1)
        mask = mask.masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask.to(self.device)

    def get_src_key_pad(self, src):
        src = (src == PAD_TOKEN_ID).long().bool()
        return src.to(self.device)

    def get_trg_key_and_mask(self, src):
        trg_mask = self.get_mask(len(src))
        trg_key_pad_mask = self.get_src_key_pad(src)
        return trg_mask, trg_key_pad_mask


    def forward(self, enc_inp, dec_inp):
        ###########################encoder############################
        # according to attention is all you need paper the encoder embedding 
        # must be multplied by the sqrt of the embedding_size

        # ensure the input is of shape [seqlen, bs]
        # if not run enc_inp = enc_inp.t()
        pos_enc = torch.arange(0, enc_inp.shape[1]).unsqueeze(0).repeat(enc_inp.shape[0], 1).to(self.device)
        # print(f'pos_enc: {pos_enc.shape}')
        # print(f'pos_enc_self: {self.pos_enc.weight.shape}')

        enc_key_pad_mask = self.get_src_key_pad(enc_inp.t()) #[seq_len, bs]

        # print(f'enc_emb: {self.enc_emb.device}')
        # print(f'enc_inp: {enc_inp.device}')
        # print(f'sqrt: {math.sqrt(self.emb_sz).device}')
        enc_inp = self.enc_emb(enc_inp.to(self.device)) * math.sqrt(self.emb_sz)
        # enc_inp = self.pos_enc(enc_inp)
        # print(f'positional: {self.pos_enc(pos_enc).shape}')
        enc_inp = enc_inp + self.pos_enc(pos_enc)
        
        enc_inp = enc_inp.transpose(0,1).to(self.device)
        
        # ensure the input is of shape [seqlen, bs, emb_sz] after emb_enc and pos_enc
        enc_out = self.trans_enc(src=enc_inp, src_key_padding_mask=enc_key_pad_mask.t()) #[seq_len, bs, emb_sz]
        
        ###########################decoder############################
        # ensure the input is of shape [seqlen, bs]
        # if not run dec_inp = dec_inp.t()
        pos_dec = torch.arange(0, dec_inp.shape[1]).unsqueeze(0).repeat(dec_inp.shape[0], 1).to(self.device)
        
        dec_mask, dec_key_pad_mask = self.get_trg_key_and_mask(dec_inp.t())

        dec_inp = self.dec_emb(dec_inp.to(self.device)) * math.sqrt(self.emb_sz)
        # dec_inp = self.pos_dec(dec_inp)
        dec_inp = dec_inp + self.pos_dec(pos_dec)
        dec_inp = dec_inp.transpose(0,1).to(self.device)
        
        # ensure the input is of shape [seqlen, bs, emb_sz] after emb_enc and pos_enc
        dec_out = self.trans_dec(tgt=dec_inp, memory=enc_out, tgt_mask=dec_mask, 
                                 tgt_key_padding_mask=dec_key_pad_mask.t(), 
                                 memory_key_padding_mask=enc_key_pad_mask.t())
        
        dec_out  = self.decout(dec_out)

        return dec_out

    def init_weight(self):
        # it is important that the distrbution of the weights be mean and std 0
        initRange = 0.1
        self.enc_emb.weight.data.uniform_(-initRange, initRange)
        self.dec_emb.weight.data.uniform_(-initRange, initRange)

        self.decout.weight.data.uniform_(-initRange, initRange)
        self.decout.bias.data.zero_()

    def eval(self):
        self.trans_enc.eval()
        self.trans_dec.eval()

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
enc_voc_sz = vocab_size_fr
dec_voc_sz = vocab_size_eng
emb_sz = 512
n_head = 8
inp_fc = 2048
num_layers = 6

In [ ]:
model = TransformerModel(enc_voc_sz, dec_voc_sz, emb_sz, n_head, inp_fc, num_layers, device=device, dropout=0.05)

In [ ]:
model.get_mask(8)

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]], device='cuda:0')

In [ ]:
model.to(device);

##Create Dataset and Dataloader

In [ ]:
class CustomTokenizer():
    def __init__(self, lang='eng', tokenizers=(tokenizer_fr, tokenizer_eng)):
        self.tok = tokenizers
        self.lang = lang

    def tokenize(self, text):
        if self.lang == 'fr':
            tokens = self.tok[0].encode(text).tokens
        elif self.lang == 'eng':
            tokens = self.tok[1].encode(text).tokens        
        return tokens

    def __call__(self, text: Str):
        # yield self.tokenize(text)
        return self.tokenize(text)

In [ ]:
def add_bos_eos_toks(x):
    return ['<sos>'] + x + ['<eos>']

In [ ]:
tfms_fr = [ColReader('fr'), CustomTokenizer(lang='fr'), add_bos_eos_toks, Numericalize(vocab=vocab_fr)]
tfms_eng = [ColReader('eng'), CustomTokenizer, add_bos_eos_toks, Numericalize(vocab=vocab_eng)]

In [ ]:
splits = RandomSplitter()(df[:10000])

In [ ]:
dsets = Datasets(df[:10000], [tfms_fr, tfms_eng], splits=splits, dl_type=SortedDL)

In [ ]:
# dls = dsets.dataloaders(bs=4, val_bs=4, before_batch=[pad], seq_len=120)

In [ ]:
# @typedispatch
# def show_batch(x:TensorText, y: TensorText, samples, max_n=2, **kwargs):
#     items = random.sample([dsets.decode((x_, y_)) for x_, y_ in zip(x, y)], k=max_n)
#     x = [[i_ for i_ in i[0] if i_ != '<pad>'] for i in items]
#     y = [[i_ for i_ in i[1] if i_ != '<pad>'] for i in items]
#     display(pd.DataFrame({'x':x, 'y':y}))

In [ ]:
# dls.show_batch(max_n=4)

In [ ]:
# dls.valid.one_batch()[0].shape

In [ ]:
dsets[0]

In [ ]:
from torch.nn.utils.rnn import pad_sequence

In [ ]:
def pad_collate(batch):
  (xx, yy) = zip(*batch)
  x_lens = [len(x) for x in xx]
  y_lens = [len(y) for y in yy]

  xx_pad = pad_sequence(xx, batch_first=True, padding_value=PAD_TOKEN_ID)
  yy_pad = pad_sequence(yy, batch_first=True, padding_value=PAD_TOKEN_ID)

  return xx_pad, yy_pad

In [ ]:
class SortSampler(torch.utils.data.sampler.Sampler):
    def __init__(self, data_source, key):
        self.data_source, self.key = data_source, key

    def __len__(self):
        return len(self.data_source)

    def __iter__(self):
        return iter(sorted(range(len(self.data_source)), key=self.key, reverse=True))

class SortishSampler(torch.utils.data.sampler.Sampler):
    def __init__(self, data_source, key, bs):
        self.data_source, self.key, self.bs = data_source, key, bs

    def __len__(self):
        return len(self.data_source)

    def __iter__(self):
        idxs = np.random.permutation(len(self.data_source))
        sz = self.bs*50
        ck_idx = [idxs[i:i+sz] for i in range(0, len(idxs), sz)]
        sort_idx = np.concatenate([sorted(s, key=self.key, reverse=True) for s in ck_idx])
        sz = self.bs
        ck_idx = [sort_idx[i:i+sz] for i in range(0, len(sort_idx), sz)]
        max_ck = np.argmax([self.key(ck[0]) for ck in ck_idx])
        ck_idx[0], ck_idx[max_ck] = ck_idx[max_ck], ck_idx[0]
        sort_idx = np.concatenate(np.random.permutation(ck_idx[1:]))
        sort_idx = np.concatenate((ck_idx[0], sort_idx))
        return iter(sort_idx)

In [ ]:
trn_enc = [s[0] for s in dsets.train]
val_enc = [s[0] for s in dsets.valid]

In [ ]:
trn_samp = SortishSampler(trn_enc, key=lambda x: len(trn_enc[x]), bs=4)
val_samp = SortSampler(val_enc, key=lambda x: len(val_enc[x]))

In [ ]:
train_dl = torch.utils.data.DataLoader(dsets.train, batch_size=16, sampler=trn_samp, collate_fn=pad_collate)
valid_dl = torch.utils.data.DataLoader(dsets.valid, batch_size=16, sampler=val_samp, collate_fn=pad_collate)

In [ ]:
trn = next(iter(train_dl))
val = next(iter(valid_dl))

In [ ]:
trn[0].shape

torch.Size([16, 110])

In [ ]:
val[0].shape

torch.Size([16, 91])

In [ ]:
len(dsets.train)

8000

## Training

In [ ]:
class PositionalEncoding(nn.Module):
    "Encode the position with a sinusoid."
    def __init__(self, d):
        super().__init__()
        self.register_buffer('freq', 1 / (10000 ** (torch.arange(0., d, 2.)/d)))
    
    def forward(self, pos):
        inp = torch.ger(pos, self.freq)
        enc = torch.cat([inp.sin(), inp.cos()], dim=-1)
        return enc

In [ ]:
class TransformerEmbedding(nn.Module):
    "Embedding + positional encoding + dropout"
    def __init__(self, vocab_sz, emb_sz, inp_p=0.):
        super().__init__()
        self.emb_sz = emb_sz
        self.embed = nn.Embedding(vocab_sz, emb_sz)
        self.pos_enc = PositionalEncoding(emb_sz)
        self.drop = nn.Dropout(inp_p)
    
    def forward(self, inp): 
        pos = torch.arange(0, inp.size(1), device=inp.device).float()
        return self.drop(self.embed(inp) * math.sqrt(self.emb_sz) + self.pos_enc(pos))

In [ ]:
def feed_forward(d_model, d_ff, ff_p=0., double_drop=True):
    layers = [nn.Linear(d_model, d_ff), nn.ReLU()]
    if double_drop: layers.append(nn.Dropout(ff_p))
    return SequentialEx(*layers, nn.Linear(d_ff, d_model), nn.Dropout(ff_p), MergeLayer(), nn.LayerNorm(d_model))

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, n_heads, d_model, d_head=None, p=0., bias=True, scale=True, device=device):
        super().__init__()
        d_head = ifnone(d_head, d_model//n_heads)
        self.n_heads,self.d_head,self.scale = n_heads,d_head,scale
        self.q_wgt,self.k_wgt,self.v_wgt = [nn.Linear(
            d_model, n_heads * d_head, bias=bias) for o in range(3)]
        self.out = nn.Linear(n_heads * d_head, d_model, bias=bias)
        self.drop_att,self.drop_res = nn.Dropout(p),nn.Dropout(p)
        self.ln = nn.LayerNorm(d_model)
        self.device = device
        
    def forward(self, q, kv, mask=None):
        return self.ln(q + self.drop_res(self.out(self._apply_attention(q, kv, mask=mask))))
    
    def create_attn_mat(self, x, layer, bs):
        return layer(x).view(bs, x.size(1), self.n_heads, self.d_head
                            ).permute(0, 2, 1, 3)
    
    def _apply_attention(self, q, kv, mask=None):
        bs,seq_len = q.size(0),q.size(1)
        wq,wk,wv = map(lambda o: self.create_attn_mat(*o,bs),
                       zip((q,kv,kv),(self.q_wgt,self.k_wgt,self.v_wgt)))
        attn_score = wq @ wk.transpose(2,3)
        if self.scale: attn_score /= math.sqrt(self.d_head)
        if mask is not None: 
            attn_score = attn_score.float().masked_fill(mask.to(self.device), -float('inf')).type_as(attn_score)
        attn_prob = self.drop_att(F.softmax(attn_score, dim=-1))
        attn_vec = attn_prob @ wv
        return attn_vec.permute(0, 2, 1, 3).contiguous().view(bs, seq_len, -1)

In [ ]:
def get_output_mask(inp, pad_idx=1):
    # return torch.triu(inp.new_ones(inp.size(1),inp.size(1)), diagonal=1)[None,None].byte()
    return torch.triu(inp.new_ones(inp.size(1),inp.size(1)), diagonal=1)[None,None].byte().bool()
#     return ((inp == pad_idx)[:,None,:,None].long() + torch.triu(inp.new_ones(inp.size(1),inp.size(1)), diagonal=1)[None,None] != 0)

In [ ]:
class EncoderBlock(nn.Module):
    "Encoder block of a Transformer model."
    #Can't use Sequential directly cause more than one input...
    def __init__(self, n_heads, d_model, d_head, d_inner, p=0., bias=True, scale=True, double_drop=True):
        super().__init__()
        self.mha = MultiHeadAttention(n_heads, d_model, d_head, p=p, bias=bias, scale=scale, device=device)
        self.ff  = feed_forward(d_model, d_inner, ff_p=p, double_drop=double_drop)
    
    def forward(self, x, mask=None): 
        return self.ff(self.mha(x, x, mask=mask))

In [ ]:
class DecoderBlock(nn.Module):
    "Decoder block of a Transformer model."
    #Can't use Sequential directly cause more than one input...
    def __init__(self, n_heads, d_model, d_head, d_inner, p=0., bias=True, scale=True, double_drop=True, device=device):
        super().__init__()
        self.mha1 = MultiHeadAttention(n_heads, d_model, d_head, p=p, bias=bias, scale=scale, device=device)
        self.mha2 = MultiHeadAttention(n_heads, d_model, d_head, p=p, bias=bias, scale=scale, device=device)
        self.ff   = feed_forward(d_model, d_inner, ff_p=p, double_drop=double_drop)
        self.device = device
    
    def forward(self, x, enc, mask_out=None): return self.ff(self.mha2(self.mha1(x, x, mask_out), enc))

In [ ]:
class Transformer(Module):
    def __init__(self, inp_vsz, out_vsz, n_layers=6, n_heads=8, d_model=256, d_head=32, 
                 d_inner=1024, p=0.1, bias=True, scale=True, double_drop=True, pad_idx=PAD_TOKEN_ID, device=device):
        self.enc_emb = TransformerEmbedding(inp_vsz, d_model, p)
        self.dec_emb = TransformerEmbedding(out_vsz, d_model, 0.)
        args = (n_heads, d_model, d_head, d_inner, p, bias, scale, double_drop)
        self.encoder = nn.ModuleList([EncoderBlock(8, 256, 32, 1024) for _ in range(n_layers)])
        self.decoder = nn.ModuleList([DecoderBlock(*args) for _ in range(n_layers)])
        self.out = nn.Linear(d_model, out_vsz)
        self.out.weight = self.dec_emb.embed.weight
        self.pad_idx = pad_idx
        self.device=device
        
    def forward(self, inp, out):
        mask_out = get_output_mask(out, self.pad_idx)
        enc,dec = self.enc_emb(inp.to(self.device)),self.dec_emb(out.to(self.device))
        # for enc_layer in self.encoder:
        #     enc = enc_layer(enc)
        #     break
        
        # for dec_layer in self.decoder:
        #     dec = dec_layer(dec, enc, mask_out)
        #     break
        # out = compose(self.decoder)(out, enc, mask_out)
        return self.out(dec)

In [ ]:
v=next(iter(train_dl))[1][0]

In [ ]:
v.shape

torch.Size([92])

In [ ]:
tokenizer_eng.decode(v.numpy())

"<sos>Where specified individuals of a designated plan, as defined in subsection 8515(4) of the Income Tax Regulations (Regulations), have no further pension credits under a defined benefit provision of the plan, determined under subsection 8301(6) of the Regulations and for the purposes of subsection 8515(1) of the Regulations, would the Minister view this condition alone in waiving the plan's designated status under subsection 8515(2) of the Regulations?<eos>"

In [ ]:
get_output_mask(v[None], PAD_TOKEN_ID).shape

torch.Size([1, 1, 92, 92])

In [ ]:
TransformerEmbedding(vocab_size_fr, 256, 0.1)(v[None]).shape

torch.Size([1, 92, 256])

https://github.com/scoutbee/pytorch-nlp-notebooks/blob/develop/6_transformer_translation.ipynb

In [ ]:
for la in nn.ModuleList([EncoderBlock(8, 256, 32, 1024) for _ in range(6)]):
    print(la(torch.randn(16, 116, 256)).shape)
    break

torch.Size([16, 116, 256])


In [ ]:
model = Transformer(vocab_size_fr, vocab_size_eng, d_model=256)

In [ ]:
model.to(device);

In [ ]:
n = next(iter(train_dl))

In [ ]:
with torch.no_grad():
    model.eval()
    display(tokenizer_eng.decode_batch(nn.Softmax(dim=2)(model(*n)).argmax(2).cpu().detach().numpy()))

["<sos>Where specified individuals of a designated plan, as defined in subsection 8515(4) of the Income Tax Regulations (Regulations), have no further pension credits under a defined benefit provision of the plan, determined under subsection 8301(6) of the Regulations and for the purposes of subsection 8515(1) of the Regulations, would the Minister view this condition alone in waiving the plan's designated status under subsection 8515(2) of the Regulations?<eos>",
 '<sos>What impact would increased harvesting efforts in interception fisheries, as could occur without a PST agreement, have on the probability of extinction of weak Pacific Northwest stocks?<eos><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>',
 '<sos>When prompted in the East, groups 

In [ ]:
prevs = []
start = [tokenizer_eng.token_to_id('<sos>')]

In [ ]:
for _ in 20:
    display(tokenizer_eng.decode_batch(nn.Softmax(dim=2)(model(n[0][1][None], )).argmax(2).cpu().detach().numpy()))
    start.append

tensor([    0,   355,  1384,  1844,   261,   292,  5112,   271,   261,   292,
        10934,   361,   298,  1009,   305,  5093,   260,   292,  1282,  1885,
           14,   815,  1067,   949,   406,  2272,   269,   292,   295,   327,
          292,    91,   277,   419,   260,   292,  3799,   319,   857,   336,
         5434,    14,   376,   288,  4092,   260,   292, 11545,   305,  4828,
        14950,  1308,   336, 10592,  2995,    15,  4272,    33,     1,     2,
            2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
            2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
            2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
            2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
            2,     2,     2,     2,     2,     2,     2,     2,     2,     2])

In [ ]:
lr = 1e-2
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
def train():
    model.train()
    all_losses = []
    losses = 0.
    for i, data in enumerate(train_dl):
        x, y = data
        # print(f'Train_Input: {tokenizer_fr.decode_batch(x.detach().numpy().tolist())}')
        # print(f'train_output: {tokenizer_eng.decode_batch(y.detach().numpy().tolist())}')
        optimizer.zero_grad()
        out = model(x, y)
        if (i % 2500 == 1):
            print(tokenizer_eng.decode_batch(nn.Softmax(dim=2)(out).argmax(2).cpu().numpy()))
            print(' ')
        loss = loss_fn(out.view(-1, out.shape[-1]), y.view(-1).to(device))
        all_losses.append(loss.item())
        losses += loss.item()
        loss.backward()
        optimizer.step()
        if(i % 5000 == 1) and i > 0:
            print(f'The loss after {i}th minibatch is {loss.cpu().detach().numpy()}')  
    return losses/len(train_dl), all_losses

def evaluate():
    model.eval()
    all_losses = []
    losses = 0.
    with torch.no_grad():
        for i, data in enumerate(valid_dl):
            x, y = data
            # print(f'Valid_Input: {tokenizer_fr.decode_batch(x.detach().numpy().tolist())}')
            # print(f'Valid_output: {tokenizer_eng.decode_batch(y.detach().numpy().tolist())}')
            out = model(x, y)
            print(f'Eval: {tokenizer_eng.decode_batch(nn.Softmax(dim=2)(out).argmax(2).cpu().numpy())}')
            loss = loss_fn(out.view(-1, vocab_size_eng), y.view(-1).to(device))
            all_losses.append(loss.item())
            losses += loss.item()
            # if(i % 250 == 0):
            #     print(f'The val_ loss after {i}th minibatch is {loss.cpu().detach().numpy()}')    
            # break

    return losses/len(valid_dl)

In [ ]:
epochs = 2
best_model = None
best_loss = float('inf')
for epoch in range(epochs):
    train_loss, all_train_losses = train()
    val_loss = evaluate()
    print('-'*80)
    print(f'EPOCH #{epoch}')
    print(f'TRAIN LOSS: {train_loss}')
    print(f'VALID LOSS: {val_loss}')
    print('-'*80)

    if val_loss < best_loss:
        best_model = model
        best_loss = val_loss

TypeError: ignored

In [ ]:
best_loss

0.0

In [ ]:
all_train_losses

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0

In [ ]:
1 in splits[0]

True

In [ ]:
# word = 'Comment ca va'
word = df.fr[1]; word, df.eng[1]

('Où sommes-nous?', 'Who are we?')

In [ ]:
[tokenizer_fr.token_to_id('<sos>')] + tokenizer_fr.encode(word).ids + [tokenizer_fr.token_to_id('<eos>')]

[0, 576, 2528, 15, 615, 33, 1]

In [ ]:
softmax = nn.Softmax(dim=2)

In [ ]:
def translate(model, word):
    model.eval()
    
    enc_inp = torch.tensor([tokenizer_fr.token_to_id('<sos>')] + tokenizer_fr.encode(word).ids + [tokenizer_fr.token_to_id('<eos>')])[None]

    tok_ids = []

    # transpose input sentence
    enc_inp = enc_inp.t().to(device)

    # get key padding mask of enc input
    enc_key_pad_mask = model.get_src_key_pad(enc_inp) #[seq_len, bs]

    # turn  word to embedding
    enc_inp = model.enc_emb(enc_inp)

    # pass through the transformer encoder
    enc_out = model.trans_enc(src=enc_inp, src_key_padding_mask=enc_key_pad_mask.t()) #[seq_len, bs, emb_sz]


    dec_inp_token_id = [tokenizer_fr.token_to_id('<sos>')]
    

    for i in range(30):

        ###########################decoder######################
        dec_inp = torch.tensor([dec_inp_token_id[-1]])[None]
        # print(f'dec_inp: {i, dec_inp.shape}')

        # transpose decoder input
        dec_inp = dec_inp.t().to(device)
        # print(f'dec_inp_after: {i, dec_inp.shape}')

        # create masks for decoder
        dec_mask, dec_key_pad_mask = model.get_trg_key_and_mask(dec_inp)

        # turn dec input to embedding
        dec_inp = model.dec_emb(dec_inp)
        # print(f'dec_inp_after_emb: {i, dec_inp.shape}')

        with torch.no_grad():



            dec_out = model.trans_dec(tgt=dec_inp, memory=enc_out, tgt_mask=dec_mask, 
                                 tgt_key_padding_mask=dec_key_pad_mask.t(), 
                                 memory_key_padding_mask=enc_key_pad_mask.t())
            
            dec_out = softmax(dec_out)
            # print(f"dec_out: {dec_out.shape}")
            # print(f'reshaped: {dec_out[-1, :, :].shape}')
            # pred_token_id = dec_out[-1, :, :].argmax(1)
            pred_token_id = dec_out.argmax(2)
            # print(f"pred_token_id : {pred_token_id.shape}")
            dec_inp_token_id.append(pred_token_id.squeeze().item())

        if pred_token_id == tokenizer_eng.token_to_id('<eos>') or pred_token_id == tokenizer_eng.token_to_id('<pad>'):
            break
        print(dec_inp_token_id)

        # if i == 3: break
    
    return tokenizer_eng.decode(dec_inp_token_id)

In [ ]:
def translate(model, word):
    model.eval()
    
    # enc_inp = torch.tensor([tokenizer_fr.token_to_id('<sos>')] + tokenizer_fr.encode(word).ids + [tokenizer_fr.token_to_id('<eos>')])[None]

    # tok_ids = []

    # # transpose input sentence
    # enc_inp = enc_inp.t().to(device)

    # # get key padding mask of enc input
    # enc_key_pad_mask = model.get_src_key_pad(enc_inp) #[seq_len, bs]

    # # turn  word to embedding
    # enc_inp = model.enc_emb(enc_inp)

    # # pass through the transformer encoder
    # enc_out = model.trans_enc(src=enc_inp, src_key_padding_mask=enc_key_pad_mask.t()) #[seq_len, bs, emb_sz]


    # dec_inp_token_id = [tokenizer_fr.token_to_id('<sos>')]
    

    # # for i in range(30):

    # ###########################decoder######################
    # dec_inp = torch.tensor([dec_inp_token_id[-1]])[None]
    # # print(f'dec_inp: {i, dec_inp.shape}')

    # # transpose decoder input
    # dec_inp = dec_inp.t().to(device)
    # # print(f'dec_inp_after: {i, dec_inp.shape}')

    # # create masks for decoder
    # dec_mask, dec_key_pad_mask = model.get_trg_key_and_mask(dec_inp)

    # # turn dec input to embedding
    # dec_inp = model.dec_emb(dec_inp)
    # # print(f'dec_inp_after_emb: {i, dec_inp.shape}')

    with torch.no_grad():
        for x, y in progress_bar(train_dl):

            dec_out = model(x, y)
            
            dec_out = softmax(dec_out)
            # print(f"dec_out: {dec_out.shape}")
            # print(f'reshaped: {dec_out[-1, :, :].shape}')
            # pred_token_id = dec_out[-1, :, :].argmax(1)
            pred_token_id = dec_out.argmax(2)
            # print(pred_token_id.shape)
            print(tokenizer_eng.decode_batch(pred_token_id.cpu().numpy()))
            break
            # print(f"pred_token_id : {pred_token_id.shape}")
            # dec_inp_token_id.append(pred_token_id.squeeze().item())

    # if pred_token_id == tokenizer_eng.token_to_id('<eos>') or pred_token_id == tokenizer_eng.token_to_id('<pad>'):
    #     break
        # print(dec_inp_token_id)

        # if i == 3: break
    
    # return tokenizer_eng.decode(dec_inp_token_id)

In [ ]:
def forward(self, inp, out):
        mask_out = get_output_mask(out, self.pad_idx)
        enc,out = self.enc_emb(inp.to(self.device)),self.dec_emb(out.to(self.device))
        for enc_layer in self.encoder:
            enc = enc_layer(enc)
        
        for dec_layer in self.decoder:
            dec = dec_layer(out, enc, mask_out)
        # out = compose(self.decoder)(out, enc, mask_out)
        return self.out(out)

In [ ]:
def translate(best_model, word):
    model.eval()
    enc_inp = torch.tensor([tokenizer_fr.token_to_id('<sos>')] + tokenizer_fr.encode(word).ids + [tokenizer_fr.token_to_id('<eos>')])[None]

    # tok_ids = []
    print(torch.tensor(enc_inp).shape)

    enc = model.enc_emb(enc_inp.to(device))

    for enc_layer in model.encoder:
        enc = enc_layer(enc)

    inps = [tokenizer_fr.token_to_id('<sos>')]
    
    for _ in range(30):
        print(torch.tensor(inps[-1])[None].shape)
        dec_inp = model.dec_emb(torch.tensor(inps[-1])[None][None].to(device))
        for dec_layer in model.decoder:
            out = dec_layer(dec_inp, enc)
    
        fin_out = model.out(out)

        # print(fin_out.shape)
        fin_out = softmax(fin_out)

        fin_out = fin_out.argmax(2)

        inps.append(fin_out.cpu().squeeze(1))
        print(fin_out.cpu().squeeze(1).squeeze().tolist())
        
        # break

In [ ]:
translate(best_model, 'D’où venez-vous?')

torch.Size([1, 9])
torch.Size([1])
0
torch.Size([1, 1])
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


RuntimeError: ignored

In [ ]:
'D’où venez-vous?'

In [ ]:
[0][-1]

0

In [ ]:
import pdb

In [ ]:
class Test(Callback):
    def begin_batch(self):
        pdb.set_trace()
        self.learn.xb = (self.xb[0], self.yb[0])
        print(self.learn.xb)

In [ ]:
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat, cbs=[Test])

In [ ]:
learn.lr_find()

KeyboardInterrupt: ignored

In [ ]:
# class TranslateDataset(torch.utils.data.Dataset):
#     def __init__(self, prll_data:pd.DataFrame):
#         self.prll_data = prll_data

#     def __len__(self):
#         return len(self.prll_data)
    
#     def __getitem__(self, i):
#        res = self.prll_data.loc[i]
#        x, y = res[0], res[1]
#     #    return padd(x, True, PAD_TOKEN).to(device), padd(y, PAD_TOKEN).to(device)
#     #    return padd(x, True, 1).to(device), padd(y, True, 1).to(device)
#        return x, y

In [ ]:
# dset = TranslateDataset(df)

In [ ]:
# def custom_collate(batch):
#     x = [item[0] for item in batch]
#     print(x)
#     y = [item[1] for item in batch]
#     return padd(x, True, 1).to(device), padd(y, True, 1).to(device)

In [ ]:
# dls = torch.utils.data.DataLoader(dset, batch_size=4, shuffle=True, collate_fn=custom_collate)